In [6]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

In [7]:
# FUNZIone per estrarre i massimi con fit parabolico
def max_fit_parabolic(x, λcenter, a, IMAX): # -a*(x-λcenter)**2 + IMAX
    return -a*(x-λcenter)**2 + IMAX

In [8]:
filename = './data/rodamina_ass_05_g1.TXT'

ass_rodamina = pd.read_csv(filename,sep='\t',header=None,skiprows=19,nrows=208) # 630 - 423 nm
ass_rodamina.columns = ['λ','A']

filename = './data/fondo acqua.TXT'

fondo_acqua = pd.read_csv(filename,sep='\t',header=None,skiprows=19,nrows=251)
fondo_acqua.columns = ['λ','A']
fondo_acqua = fondo_acqua[(fondo_acqua['λ'] <= 630) & (fondo_acqua['λ'] >= 423)]

In [9]:
fig = go.Figure()

# Add the measured absorbance spectrum
fig.add_trace(go.Scatter(
    x=ass_rodamina['λ'],
    y=ass_rodamina['A'],
    mode='lines',
    name='Signal',
    opacity=0.3
))

fig.add_trace(go.Scatter(
    x=fondo_acqua['λ'],
    y=fondo_acqua['A'],
    mode='lines',
    name='Background',
    opacity=0.3
))

fig.add_trace(go.Scatter(
    x=fondo_acqua['λ'],
    y=ass_rodamina['A'] - fondo_acqua['A'],
    mode='lines',
    name='Spectrum Rhodamine 6G',
    line=dict(width=4)  # Make the line thicker
))


# Update layout for the plot
fig.update_layout(
    # title='Absorbance Spectrum of Rodamine',
    xaxis_title='Wavelength (nm)',
    yaxis_title='Absorbance',
    legend=dict(x=1, y=1, xanchor='right', yanchor='top'),  # Position legend at the top right
    font=dict(size=14),
    height=600,
    width=800
)

fig.write_html(dir_path +r"\html\absorbance_spectrum.html")
fig.write_image(dir_path + r"\images\absorbance_spectrum.png")
fig.show()

In [10]:
λ_ass = ass_rodamina['λ'].to_numpy()
A_rod = ass_rodamina['A'].to_numpy() - fondo_acqua['A'].to_numpy()

l0 = ass_rodamina['A'].idxmax()

popt, pcov = curve_fit(max_fit_parabolic, λ_ass[l0-5:l0+5], A_rod[l0-5:l0+5], p0=[530, 2, 0.5])
λcenter, a, A = popt
error_λcenter, error_a, error_A = np.sqrt(np.diag(pcov))

print('Picco di assorbimento A: ',final_val(A,error_A,decimals=4))
print(A,error_A)

Picco di assorbimento A:  0.48 ± 0.0003 
0.4800416799849726 0.0003353431037236748
